In [1]:
import time
from threading import Thread, Lock
from math import inf

In [13]:
def compare(word, solution):
    response = 0
    disabled_pos = []
    for w in range(len(word)):
        response = response * 10
        if word[w] == solution[w]:
            response = response + 2
            disabled_pos.append(w)
        elif word[w] in [i if w not in disabled_pos else '0' for (w,i) in enumerate(solution)]:
            response = response + 1
    return response

def get_sets(word, remaining):
    combs = {}
    for solution in remaining:
        score = compare(word, solution)
        combs.setdefault(score, [])
        combs.setdefault(score, []).append(solution)
    return combs

def filter_set(remaining, word, response):
    return [r for r in remaining if compare(word, r) == response]

In [14]:
# Each node need to know: remaining feasible set
# Optionally: parent, and current move counter
class Node:
    def __init__(self, parent, remaining, counter, info):
        self.parent = parent
        self.children = {}
        self.child_scores = {}
        self.remaining = remaining
        self.counter = counter
        self.expanded = False
        self.finished = False
        self.pruned = False
        self.avg_moves = None
        self.best_move = ''
        self.local_bound = inf
        self.info = info
        self.pruned_children = []
        self.lock = Lock()
        # print(f"New Node: {parent} {counter} {info} {remaining}")

    def get_status(self):
        return {
            'id': id(self),
            'parent': self.parent,
            'remaining': self.remaining,
            'counter': self.counter,
            'expanded': self.expanded,
            'finished': self.finished,
            'pruned': self.pruned,
            'avg_moves': self.avg_moves,
            'children': len(self.children),
            'best_move': self.best_move,
            'info': self.info
        }

    def check(self, global_bound):
        if self.pruned:
            return
        with self.lock:
            score = {}
            completed = True
            if len(self.children) > 0:
                set_size = len(self.remaining)
                for word, word_children in self.children.items():
                    if word in self.pruned_children:
                        continue
                    if self.child_scores.get(word) is not None:
                        continue
                    word_done = True
                    # print(word, word_children)
                    for response, response_child in word_children.items():
                        if not response_child.finished:
                            word_done = False
                            completed = False
                            continue

                        if response_child.finished and response_child.avg_moves is None:
                            word_done = False
                            self.pruned_children.append(word)
                            break
                        else:
                            child_score = len(response_child.remaining) / set_size * response_child.avg_moves
                            score[word] = score.get(word, 0) + child_score
                    if word_done:
                        # print(f"New value added: {self.info} {word} {score[word]}")
                        self.child_scores[word] = score[word]
                        self.local_bound = min(self.local_bound, score[word])
                        # print(f"LOCAL BOUND UPDATED: {self.info} by {word}: {self.local_bound}")

            score = self.child_scores
            for word in score:
                if word in self.pruned_children:
                    continue # already pruned
                if score[word] > global_bound or score[word] > self.local_bound: # already worse than best case
                    # print(f"PRUNING CHILD NODE: {self.info} for word {word} score {score[word]} bound {self.local_bound} and {global_bound}")
                    for response_child in self.children[word].values():
                        response_child.prune()
                    self.pruned_children.append(word)

        # if len(self.children) == 0 and self.counter > 1:
        #     self.prune()
        #     if self.parent:
        #         self.parent.check(global_bound)
        # else:
        #     self.finished = True
        #     return

        if completed:
            self.finished = True
            if len(score) == 0:
                self.prune()
                if self.parent:
                    self.parent.check(global_bound)
                return
            best_word = min(score, key=score.get)
            self.avg_moves = score[best_word]
            self.best_move = best_word
            # if self.counter <= 2:
                # print("COMPLETED")
                # print(self.get_status())
                # print(score)
            if self.parent:
                self.parent.check(min(self.local_bound, global_bound))

    def prune(self):        
        if self.pruned:
            return
        with self.lock:
            # print(f"PRUNED {self.info}")
            self.pruned = True
            self.finished = True
            for word, word_children in self.children.items():
                for response, response_child in word_children.items():
                    if not response_child.finished:
                        response_child.prune()

    def set_solved(self):
        self.finished = True
        self.avg_moves = self.parent.counter

In [15]:
# Each worker picks up next available job possible and expands the set
class Worker:
    def __init__(self, id, controller):
        self.id = id
        self.controller = controller
    
    def __call__(self):
        print(f"Start worker {self.id}")
        while self.controller.has_more():
            self.run()
        print(f"End worker: {self.id}")

    def run(self):
        node = self.controller.get_next()
        if node is None:
            time.sleep(0.1)
            return

        cutoff = self.controller.options['cutoff']
        if node.counter > cutoff:
            node.finished = True
            node.avg_moves = cutoff + 1
            node.parent.check(self.controller.get_bound())
            return
            # node.prune()
            # node.parent.check(self.controller.get_bound())
            return

        if node.pruned:
            # node.finished = True
            # node.avg_moves = None
            node.parent.check(self.controller.get_bound())
            return

        if len(node.remaining) == 1:
            node.finished = True
            node.avg_moves = node.counter
            node.best_move = node.remaining[0]
            # print(node.get_status())
            parent = node.parent
            if parent:
                parent.check(self.controller.get_bound())
            return

        search_space = self.controller.get_search_space(node)
        for word, sets in search_space:
            if word in node.info:
                continue
            # sets = get_sets(word, node.remaining)
            children_dict = node.children.setdefault(word, dict())
            # print(word, sets)
            for response in sets:
                child = Node(node, sets[response].copy(), node.counter + 1, node.info.copy() + [word, response])
                children_dict[response] = child
                if response == 22222:
                    child.set_solved()
                else:
                    self.controller.add_node(child)
            node.expanded = True


In [16]:
class Controller():

    def __init__(self, first_node, options):
        self.active_nodes = [first_node]
        self.first_node = first_node
        self.global_bound = inf
        self.options = options
        self.node_lock = Lock()
        # self.workers = [Worker(1, self)]

    def start(self):
        if self.options['threaded']:
            workers = [Worker(i, self) for i in range(15)]
            threads = [Thread(target=w) for w in workers]
            for t in threads:
                t.start()
            for t in threads:
                t.join()
        else:
            w = Worker(1, self)
            w.__call__()

    def get_search_space(self, node):
        # selected_words = []
        main_set = self.options['selection']
        remaining_only = self.options['remaining_only']
        filter_rule = self.options['filter_rule']
        
        if remaining_only:
            main_set = node.remaining
        if filter_rule[0] == 'top':
            all_sets = tuple(get_sets(word, node.remaining) for word in main_set)
            scoring = [(max(len(j) for j in i.values()), -len([j for j in i.values() if len(j) == 1]), 0 if i in node.remaining else 1) for i in all_sets]
            topN = set(sorted(scoring)[0:filter_rule[1]])
            pairs = tuple(zip(main_set, all_sets))
            chosen_pairs = (pairs[i] for i in range(len(main_set)) if scoring[i] in topN)
            return list(chosen_pairs)[0:filter_rule[1]]
        elif filter_rule[0] == 'none':
            return zip(main_set, [get_sets(w, node.remaining) for w in main_set])

    def add_node(self, node):
        with self.node_lock:
            self.active_nodes.append(node)

    def get_next(self, method='smallest_set'):
        with self.node_lock:
            self.active_nodes = [i for i in self.active_nodes if not i.pruned and not i.finished]
        if len(self.active_nodes) > 0:
            # ordered
            if method == 'ordered':
                with self.node_lock:
                    node = self.active_nodes.pop(0)
            elif method == 'smallest_set':
                with self.node_lock:
                    set_sizes = [[-i.counter, len(i.remaining)] for i in self.active_nodes]
                    min_index = set_sizes.index(min(set_sizes))
                    node = self.active_nodes.pop(min_index)
            # print(f"Processing {id(node)}")
            return node
        else:
            return None

    def get_bound(self):
        return self.global_bound

    def update_bound(self, value):
        self.global_bound = min(self.global_bound, value)

    def has_more(self):
        with self.node_lock:
            print(f"Remaining nodes: {len(self.active_nodes)}")
            if len(self.active_nodes) == 0:
                self.first_node.check(self.global_bound)
            # print(self.first_node.get_status())
            return len(self.active_nodes) > 0 or not self.first_node.finished


In [17]:
with open("solutions.txt") as f:
    text = f.read()
    solution_set = [i.replace('"', '').replace(' ', '') for i in text.split(",")]
solution_set[0:5]

['cigar', 'rebut', 'sissy', 'humph', 'awake']

In [18]:
with open("nonsolutions.txt") as f:
    text = f.read()
    nonsolution_set = [i.replace('"', '').replace(' ', '') for i in text.split(",")]
nonsolution_set[0:5]

['aahed', 'aalii', 'aargh', 'aarti', 'abaca']

In [19]:
# Initial try
filtered_set = filter_set(solution_set, 'raise', 1001)

In [20]:
input = nonsolution_set + solution_set
target = filtered_set

initial_node = Node(None, target, 1, ['StartNode'])
options = {'filter_rule': ('top', 5), 'selection': input, 'remaining_only': False, 'cutoff': 6, 'threaded': False}
c = Controller(initial_node, options)
t0 = time.time()
c.start()
t1 = time.time()
initial_node.get_status()

Start worker 1
Remaining nodes: 1
Remaining nodes: 141
Remaining nodes: 140
Remaining nodes: 139
Remaining nodes: 138
Remaining nodes: 137
Remaining nodes: 136
Remaining nodes: 135
Remaining nodes: 134
Remaining nodes: 133
Remaining nodes: 132
Remaining nodes: 131
Remaining nodes: 130
Remaining nodes: 129
Remaining nodes: 128
Remaining nodes: 127
Remaining nodes: 126
Remaining nodes: 125
Remaining nodes: 124
Remaining nodes: 123
Remaining nodes: 122
Remaining nodes: 121
Remaining nodes: 120
Remaining nodes: 119
Remaining nodes: 118
Remaining nodes: 117
Remaining nodes: 116
Remaining nodes: 115
Remaining nodes: 114
Remaining nodes: 113
Remaining nodes: 112
Remaining nodes: 111
Remaining nodes: 110
Remaining nodes: 109
Remaining nodes: 108
Remaining nodes: 107
Remaining nodes: 106
Remaining nodes: 105
Remaining nodes: 104
Remaining nodes: 103
Remaining nodes: 102
Remaining nodes: 101
Remaining nodes: 100
Remaining nodes: 99
Remaining nodes: 98
Remaining nodes: 97
Remaining nodes: 96
Rema

{'id': 2293187748864,
 'parent': None,
 'remaining': ['heath',
  'death',
  'delta',
  'kebab',
  'enema',
  'cheat',
  'peach',
  'abbey',
  'pleat',
  'ahead',
  'fella',
  'metal',
  'beady',
  'glean',
  'eclat',
  'equal',
  'leaky',
  'exalt',
  'decay',
  'clean',
  'ocean',
  'alley',
  'leafy',
  'delay',
  'meant',
  'fecal',
  'beach',
  'enact',
  'heady',
  'theta',
  'amend',
  'begat',
  'penal',
  'adept',
  'medal',
  'meaty',
  'bleak',
  'cheap',
  'decal',
  'pedal',
  'bleat',
  'heavy',
  'cleat',
  'agent',
  'annex',
  'omega',
  'leant',
  'abled',
  'legal',
  'began',
  'hyena',
  'angel',
  'petal',
  'fetal',
  'leapt',
  'teach',
  'apnea',
  'tweak',
  'mealy',
  'wheat',
  'dealt',
  'gleam',
  'pecan',
  'plead',
  'vegan',
  'exact',
  'knead',
  'mecca',
  'leach'],
 'counter': 1,
 'expanded': True,
 'finished': True,
 'pruned': False,
 'avg_moves': 2.7536231884057982,
 'children': 5,
 'best_move': 'tepal',
 'info': ['StartNode']}

In [21]:
target = filter_set(target, 'tepal', 1010)
initial_node = Node(None, target, 1, ['StartNode'])
options = {'filter_rule': ('top', 5), 'selection': input, 'remaining_only': False, 'cutoff': 6, 'threaded': False}
c = Controller(initial_node, options)
t0 = time.time()
c.start()
t1 = time.time()
initial_node.get_status()

Start worker 1
Remaining nodes: 1
Remaining nodes: 30
Remaining nodes: 29
Remaining nodes: 28
Remaining nodes: 27
Remaining nodes: 26
Remaining nodes: 25
Remaining nodes: 24
Remaining nodes: 23
Remaining nodes: 22
Remaining nodes: 21
Remaining nodes: 20
Remaining nodes: 19
Remaining nodes: 18
Remaining nodes: 17
Remaining nodes: 16
Remaining nodes: 15
Remaining nodes: 14
Remaining nodes: 13
Remaining nodes: 12
Remaining nodes: 11
Remaining nodes: 10
Remaining nodes: 9
Remaining nodes: 8
Remaining nodes: 7
Remaining nodes: 6
Remaining nodes: 5
Remaining nodes: 4
Remaining nodes: 3
Remaining nodes: 2
Remaining nodes: 1
Remaining nodes: 0
End worker: 1


{'id': 2293190768336,
 'parent': None,
 'remaining': ['enema', 'abbey', 'amend', 'annex', 'omega', 'hyena'],
 'counter': 1,
 'expanded': True,
 'finished': True,
 'pruned': False,
 'avg_moves': 1.9999999999999998,
 'children': 5,
 'best_move': 'abamp',
 'info': ['StartNode']}

In [22]:
target = filter_set(target, 'abamp', 22000)
initial_node = Node(None, target, 1, ['StartNode'])
options = {'filter_rule': ('top', 5), 'selection': input, 'remaining_only': False, 'cutoff': 6, 'threaded': False}
c = Controller(initial_node, options)
t0 = time.time()
c.start()
t1 = time.time()
initial_node.get_status()

Start worker 1
Remaining nodes: 1
Remaining nodes: 0
End worker: 1


{'id': 2293190774208,
 'parent': None,
 'remaining': ['abbey'],
 'counter': 1,
 'expanded': False,
 'finished': True,
 'pruned': True,
 'avg_moves': 1,
 'children': 0,
 'best_move': 'abbey',
 'info': ['StartNode']}